# 性能与显存可视化
读取 `tests/bench.py` 的输出csv并绘制：
- 两组（对照/实验）的时延对比（平均/中位）
- 峰值显存对比

In [6]:
import json
import os
from typing import List, Dict

import pandas as pd
import altair as alt

# 显存 byte 转 MB
def bytes_to_mb(x):
    return x / (1024 * 1024)

# 生成横向柱状图：时延（avg/p50）
def plot_latency_horizontal(df_latency: pd.DataFrame):
    chart = alt.Chart(df_latency).mark_bar().encode(
        y=alt.Y('metric:N', title='指标', sort=None),
        x=alt.X('ms:Q', title='时延 (ms)'),
        color='metric:N',
        column=alt.Column('bits:N', title='位宽 (bits)')
    ).properties(title='对照组 vs 实验组 的时延对比（avg/p50）')
    return chart

# 生成横向柱状图：峰值显存（MB）
def plot_memory_horizontal(df_memory: pd.DataFrame):
    chart = alt.Chart(df_memory).mark_bar().encode(
        y=alt.Y('metric:N', title='指标', sort=None),
        x=alt.X('MB:Q', title='峰值显存 (MB)'),
        color='metric:N',
        column=alt.Column('bits:N', title='位宽 (bits)')
    ).properties(title='对照组 vs 实验组 的峰值显存对比（MB）')
    return chart

# 准备数据（宽→长），并转 MB
_df = pd.read_csv("results.csv")
_df_mem = _df.copy()
_df_mem['ctrl_peak_mem_mb'] = _df_mem['ctrl_peak_mem'].map(bytes_to_mb)
_df_mem['exp_peak_mem_mb'] = _df_mem['exp_peak_mem'].map(bytes_to_mb)

df_long_latency2 = pd.melt(
    _df,
    id_vars=["bits", "batch_size", "num_heads", "seq_len", "head_dim", "fp16_ratio", "causal", "iters", "warmup"],
    value_vars=["ctrl_avg_ms", "exp_avg_ms", "ctrl_p50_ms", "exp_p50_ms"],
    var_name="metric",
    value_name="ms",
)

df_long_mem2 = pd.melt(
    _df_mem,
    id_vars=["bits", "batch_size", "num_heads", "seq_len", "head_dim", "fp16_ratio", "causal", "iters", "warmup"],
    value_vars=["ctrl_peak_mem_mb", "exp_peak_mem_mb"],
    var_name="metric",
    value_name="MB",
)

plot_latency_horizontal(df_long_latency2) & plot_memory_horizontal(df_long_mem2)


alt.VConcatChart(...)